In [53]:
mnist_results_paths = ['../output/natural-mnist-results.json',
                       '../output/ablation-mnist-results.json',
                       '../output/ablation-mnist-results2.json',
                       '../output/ablation-mnist-results3.json',
                       '../output/ablation-mnist-results-spsa.json', 
                       '../output/maxfit-sensitivity-mnist-results.json',
                       '../output/natural-maxfit-sensitivity-mnist-results.json',
                       '../output/maxfit-sensitivity-mnist-results-spsa.json']
cifar10_results_paths = ['../output/natural-cifar10-results.json',
                         '../output/natural-cifar10-results2.json',
                         '../output/ablation-cifar10-results.json',
                         '../output/ablation-cifar10-results2.json',
                         '../output/natural-cifar10-results4.json',
                         '../output/ablation-cifar10-results4.json',
                         '../output/manual-evaluation.json',]

In [3]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
import os
from scipy import stats

In [65]:
import json

def read_json(path):
    rows = []
    with open(path) as f:
        for line in f:
            if line.strip():
                row = json.loads(line[:-1])
                if 'attack' in row:
                    row['attack_type'] = row['attack']
                    del row['attack']
                rows.append(row)
    df = pd.DataFrame(rows)
    if 'eps' in df.columns:
        df = df.drop_duplicates(['model_path', 'attack_type', 'eps'], keep='last')
    return df

def read_jsons(paths):
    dfs = [read_json(path) for path in paths if os.path.exists(path)]
    return pd.concat(dfs, sort=False).reset_index()

In [5]:
def process(df):
    df['attack_type'] = df.apply(
        lambda r: ('%s(eps=%.1f)' % (r['attack_type'], r['eps'])
                   if 'eps' in r else r['attack_type']), axis=1)
    df['model_name'] = df.model_path.apply(os.path.basename)
    counts = df.groupby(['model_name', 'attack_type']).agg({'accuracies': len})
    duplicates = counts[counts.accuracies >= 2]
    if len(duplicates) > 0:
        print(duplicates.reset_index().values)
    assert len(duplicates) == 0, 'Please remove duplicates'
    return df

# MNIST results

In [65]:
results = process(read_jsons(mnist_results_paths))

In [12]:
def compute_accuracy_and_ttest(col):
    if col.name in ('index', 'model_name'):
        return list(col)
    formatted_accuracies = []
    for prev, curr in zip([None] + list(col), list(col)):
        if isinstance(curr, (str, float)):
            formatted_accuracies.append(curr)
        elif isinstance(curr, list):
            curr = np.around(curr, 2)
            acc = np.mean(curr)
            acc = '%.2f' % acc
            if isinstance(prev, list):
                prev = np.around(prev, 2)
                _, p_value = stats.ttest_ind(prev, curr, equal_var = False)
                acc += '*' if p_value < 0.05 else '<'
            else:
                acc += '<'
            formatted_accuracies.append(acc)
        else:
            raise ValueError("Unsupported type: " + str(curr))
    return pd.Series(formatted_accuracies)

In [13]:
def format_metrics(df, formatted_names, model_order, attack_order, values_col):
    # filtering
    df = df[df.model_name.isin(formatted_names) &
            df.attack_type.isin(attack_order)].copy()
    # convert to wide format
    df = df.pivot(index='model_name', values=values_col, columns='attack_type')
    # change column order
    df = (df.reindex(sorted(df.columns, key=attack_order.__getitem__), axis=1)
          .reset_index().fillna('-'))
    df['model_order'] = df.model_name.apply(model_order.__getitem__)
    df = df.sort_values(['model_order']).drop(columns='model_order')
    # compute mean accuracy and p-values
    df = df.apply(compute_accuracy_and_ttest)
    # format model names last
    df['model_name'] = df.model_name.apply(formatted_names.__getitem__)    
    return df

def format_table(df, formated_names, model_order, attack_order):
    df = format_metrics(df, formatted_names, model_order, attack_order, values_col='accuracies')
    latex = df.to_latex()
    latex = latex.replace('*', '$^*$')
    latex = latex.replace('<', '\hspace{0.5em}')
    return df, latex

## Ablation table

In [68]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog (beta=2)'), 
    ('relog-maxout_4.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout_2_4.pkl', '+ MinOut (k=2)'),
    ('relog-elliptical-maxout_4.pkl', '+ Elliptical'),
    ('relog-elliptical-maxout_4-max_fit_l1_0001.pkl', '+ MaxFit (L1)'),  
    ('relog-elliptical-maxout_4-mse-max_fit_l1.pkl', '+ MSE training'),  
    ('relog-elliptical-maxout_4-mse-max_fit_l1-overlay.pkl', '+ Negative examples'),  
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [69]:
used_attacks = ['none(eps=nan)', # (natural)
                'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)', 'CW(eps=nan)',
                'BIM(eps=0.3)', 'SPSA(eps=0.3)']
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [70]:
results.attack_type.unique()

array(['none(eps=nan)', 'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)',
       'CW(eps=nan)', 'BIM(eps=0.3)', 'MaxConf(eps=0.3)', 'SPSA(eps=0.3)'],
      dtype=object)

In [71]:
ablation_table, latex = format_table(results, formatted_names, model_order, attack_order)

In [72]:
ablation_table

attack_type,model_name,none(eps=nan),FGM_inf(eps=0.3),FGM_L2(eps=2.0),CW(eps=nan),BIM(eps=0.3),SPSA(eps=0.3)
7,Baseline,0.99<,0.10<,0.71<,0.05<,0.00<,0.16<
6,+ ReLog (beta=2),0.99<,0.37*,0.85*,0.23*,0.03*,0.44*
4,+ MaxOut (k=4),0.99<,0.40*,0.85<,0.23<,0.04<,0.42<
5,+ MinOut (k=2),0.99<,0.67*,0.91*,0.32*,0.17*,0.46*
3,+ Elliptical,0.96*,0.56*,0.71*,0.59*,0.07*,0.34*
0,+ MaxFit (L1),0.97*,0.53*,0.75*,0.67*,0.07<,0.33<
2,+ MSE training,0.97<,0.35*,0.79*,0.33*,0.07<,0.37*
1,+ Negative examples,0.94*,0.03*,0.26*,0.84*,0.00*,0.01*


In [73]:
print(latex)

\begin{tabular}{llllllll}
\toprule
attack\_type &           model\_name & none(eps=nan) & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) & SPSA(eps=0.3) \\
\midrule
7 &             Baseline &         0.99\hspace{0.5em} &            0.10\hspace{0.5em} &           0.71\hspace{0.5em} &       0.05\hspace{0.5em} &        0.00\hspace{0.5em} &         0.16\hspace{0.5em} \\
6 &     + ReLog (beta=2) &         0.99\hspace{0.5em} &            0.37$^*$ &           0.85$^*$ &       0.23$^*$ &        0.03$^*$ &         0.44$^*$ \\
4 &       + MaxOut (k=4) &         0.99\hspace{0.5em} &            0.40$^*$ &           0.85\hspace{0.5em} &       0.23\hspace{0.5em} &        0.04\hspace{0.5em} &         0.42\hspace{0.5em} \\
5 &       + MinOut (k=2) &         0.99\hspace{0.5em} &            0.67$^*$ &           0.91$^*$ &       0.32$^*$ &        0.17$^*$ &         0.46$^*$ \\
3 &         + Elliptical &         0.96$^*$ &            0.56$^*$ &           0.71$^*$ &       0.59$^*$ &     

## Check a few entries

In [15]:
def print_mean_acc(attack, model):
    row = results[(results.attack_type == attack) & (results.model_name == model)]
    print(np.mean(row.accuracies.iloc[0]))

In [39]:
print_mean_acc('FGM_L2(eps=2.0)', 'relog-elliptical-maxout_4-mse-max_fit_l1-overlay.pkl')

0.26130000000000003


In [42]:
print_mean_acc('BIM(eps=0.3)', 'relog-elliptical-maxout_4-max_fit_l1.pkl')

0.0131


In [41]:
print_mean_acc('SPSA(eps=0.3)', 'relu.pkl')

0.16133333333333336


## Check the effect of L1

In [52]:
results[results.model_path.str.contains('sensitivity') & results.attack_type.str.contains('CW')][['model_name', 'accuracies']]

,model_name,accuracies
54,relog-elliptical-maxout_4-max_fit_l1_02.pkl,"[0.25, 0.2, 0.21, 0.21, 0.34, 0.19, 0.2, 0.18,..."
58,relog-elliptical-maxout_4-max_fit_l1_0001.pkl,"[0.63, 0.66, 0.62, 0.68, 0.7, 0.66, 0.58, 0.63..."
62,relog-elliptical-maxout_4-max_fit_l1_001.pkl,"[0.59, 0.59, 0.48, 0.53, 0.47, 0.5, 0.49, 0.59..."
66,relog-elliptical-maxout_4-max_fit_l1_03.pkl,"[0.28, 0.17, 0.32, 0.27, 0.15, 0.28, 0.26, 0.2..."
70,relog-elliptical-maxout_4-max_fit_l1_002.pkl,"[0.56, 0.65, 0.55, 0.48, 0.55, 0.55, 0.49, 0.4..."
74,relog-elliptical-maxout_4-max_fit_l1_05.pkl,"[0.35, 0.46, 0.45, 0.34, 0.37, 0.37, 0.43, 0.3..."


In [54]:
results[results.model_name.str.contains('max_fit_l1_0001')][['model_name', 'attack_type', 'accuracies']]

,model_name,attack_type,accuracies
56,relog-elliptical-maxout_4-max_fit_l1_0001.pkl,FGM_inf(eps=0.3),"[0.6, 0.5, 0.52, 0.59, 0.53, 0.51, 0.54, 0.51,..."
57,relog-elliptical-maxout_4-max_fit_l1_0001.pkl,FGM_L2(eps=2.0),"[0.77, 0.74, 0.72, 0.76, 0.77, 0.72, 0.79, 0.8..."
58,relog-elliptical-maxout_4-max_fit_l1_0001.pkl,CW(eps=nan),"[0.63, 0.66, 0.62, 0.68, 0.7, 0.66, 0.58, 0.63..."
59,relog-elliptical-maxout_4-max_fit_l1_0001.pkl,BIM(eps=0.3),"[0.07, 0.09, 0.03, 0.07, 0.05, 0.1, 0.07, 0.06..."


I have the feeling that my regularization formula is not entirely right... Elliptical used to get highest results but after some changes it suddenly perform oddly.

# CIFAR-10 results

In [101]:
cifar10_results = process(read_jsons(cifar10_results_paths))

In [102]:
cifar10_results.attack_type.unique()

array(['none(eps=nan)', 'FGM_L2(eps=2.0)', 'FGM_inf(eps=0.3)',
       'BIM(eps=0.3)', 'CW(eps=nan)'], dtype=object)

In [103]:
cifar10_results.model_name.unique()

array(['relu.pkl', 'relog.pkl', 'relog-elliptical.pkl',
       'relog-elliptical-maxfit.pkl',
       'relog-elliptical-maxout-maxfit_l1.pkl',
       'relog-maxout-elliptical-max_fit_l1-mse.pkl',
       'relog-maxout-elliptical-max_fit_l1-mse-overlay.pkl',
       'relog-maxout.pkl',
       'relog-minmaxout-elliptical_stop1-maxfit-mse-overlay.pkl',
       'relog-minmaxout.pkl',
       'relog-minmaxout-elliptical_stop1-maxfit.pkl',
       'relog-minmaxout-elliptical_stop1.pkl',
       'relog-minmaxout-elliptical_stop1-maxfit-mse.pkl'], dtype=object)

In [104]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog'), 
    ('relog-maxout.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout.pkl', '+ MinOut (k=2)'),
    ('relog-minmaxout-elliptical_stop1.pkl', '+ Elliptical'),
    ('relog-minmaxout-elliptical_stop1-maxfit.pkl', '+ MaxFit (L1)'),
    ('relog-minmaxout-elliptical_stop1-maxfit-mse.pkl', '+ MSE training'),
    ('relog-minmaxout-elliptical_stop1-maxfit-mse-overlay.pkl', '+ Negative examples')
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [105]:
used_attacks = ['none(eps=nan)', # (natural)
                'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)', 'CW(eps=nan)',
                'BIM(eps=0.3)']
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [106]:
ablation_table, latex = format_table(cifar10_results, formatted_names, model_order, attack_order)

In [107]:
ablation_table

attack_type,model_name,none(eps=nan),FGM_inf(eps=0.3),FGM_L2(eps=2.0),CW(eps=nan),BIM(eps=0.3)
7,Baseline,0.75<,0.08<,0.42<,0.00<,0.01<
6,+ ReLog,0.74*,0.17*,0.44*,0.03*,0.01<
0,+ MaxOut (k=4),0.66*,0.13*,0.34*,0.01*,0.00*
5,+ MinOut (k=2),0.72*,0.21*,0.38*,0.03*,0.01*
4,+ Elliptical,0.75*,0.18*,0.42*,0.04*,0.01*
3,+ MaxFit (L1),0.77*,0.20*,0.45*,0.05*,0.02*
2,+ MSE training,0.68*,0.15*,0.36*,0.04*,0.01*
1,+ Negative examples,0.43*,0.14*,0.21*,0.04<,0.01<


In [108]:
print(latex)

\begin{tabular}{lllllll}
\toprule
attack\_type &           model\_name & none(eps=nan) & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) \\
\midrule
7 &             Baseline &         0.75\hspace{0.5em} &            0.08\hspace{0.5em} &           0.42\hspace{0.5em} &       0.00\hspace{0.5em} &        0.01\hspace{0.5em} \\
6 &              + ReLog &         0.74$^*$ &            0.17$^*$ &           0.44$^*$ &       0.03$^*$ &        0.01\hspace{0.5em} \\
0 &       + MaxOut (k=4) &         0.66$^*$ &            0.13$^*$ &           0.34$^*$ &       0.01$^*$ &        0.00$^*$ \\
5 &       + MinOut (k=2) &         0.72$^*$ &            0.21$^*$ &           0.38$^*$ &       0.03$^*$ &        0.01$^*$ \\
4 &         + Elliptical &         0.75$^*$ &            0.18$^*$ &           0.42$^*$ &       0.04$^*$ &        0.01$^*$ \\
3 &        + MaxFit (L1) &         0.77$^*$ &            0.20$^*$ &           0.45$^*$ &       0.05$^*$ &        0.02$^*$ \\
2 &       + MSE trainin